In [1]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as matim
import ipywidgets as widgets

In [6]:
img = matim.imread('./images/dolphin.jpg')

In [8]:
@widgets.interact_manual(style = ["detail", "pencil", "cartoon"])
def app(style="pencil"):
    if style == "detail":
        @widgets.interact_manual(s = (0, 200, 1), r = (0, 1, 0.1))
        def dtail(s = 100, r = 0.5):
            detailImg = cv2.detailEnhance(img, sigma_s = s, sigma_r = r)
            plt.figure(figsize = (15, 8)) 
            plt.subplot(1,2,1)
            plt.imshow(img)
            plt.yticks([]), plt.xticks([]) # 눈금을 null 값으로 넣겠다. 
            plt.title("Original Image")
            plt.subplot(1,2,2)
            plt.imshow(detailImg)
            plt.title("Dtail Image")
            plt.yticks([]), plt.xticks([])
            plt.show()
    elif style == "pencil":
        def adjust_gamma(image, gamma = 1):
            invGamma = 1.0 / gamma 
            table = np.array([
                ((i/255)**invGamma) * 255 for i in np.arange(0, 256)
            ])
            lut_img = cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))
            return lut_img
        @widgets.interact_manual(ksize=(3, 25, 2), sigmaX = (1, 15, 2), gamma = (0, 1, 0.01))
        def pencil_art_image(ksize = 21, sigmaX = 9, gamma = 0.1):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_blur = cv2.GaussianBlur(gray, (ksize, ksize), sigmaX) 
            gray_blur_divide = cv2.divide(gray, gray_blur, scale = 256)
            pencil_sketch = adjust_gamma(gray_blur_divide, gamma = gamma) 

            plt.figure(figsize = (15, 8))
            plt.subplot(1, 2, 1)
            plt.yticks([]), plt.xticks([])
            plt.title("Original Image")
            plt.imshow(img)
            plt.subplot(1, 2, 2)
            plt.yticks([]), plt.xticks([])
            plt.title("Pencil Image")
            plt.imshow(pencil_sketch, cmap = "gray")
            plt.show()
    elif style == "cartoon":
        def edge_mask(img, ksize, block_size):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_median = cv2.medianBlur(gray, ksize)
            edges = cv2.adaptiveThreshold(gray_median, 255, 
                cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, ksize)
            return edges 

        def kmeans_cluster(img, k):
            data = np.float32(img).reshape((-1, 3))

            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

            ret, label, center = cv2.kmeans(data, k, None, criteria, 
                5, cv2.KMEANS_RANDOM_CENTERS)
            center = np.uint8(center)
            result = center[label.flatten()]
            result = result.reshape(img.shape)
            return result
        @widgets.interact_manual(ksize = (3, 25, 2), 
        block_size = (1, 21, 2), k = (1, 20, 1), d = (5, 15, 2), 
        sigmacolor = (1, 300),sigmaspace = (1, 300))

        def cartoon_image(ksize = 5, 
        block_size = 7, k = 7, d = 7, sigmacolor = 200, sigmaspace = 200):
            edge_Mask = edge_mask(img, ksize, block_size)
            cluster_img = kmeans_cluster(img, k)
            bilateral = cv2.bilateralFilter(cluster_img, d=d, 
            sigmaColor=sigmacolor, sigmaSpace=sigmaspace)
            cartoon = cv2.bitwise_and(bilateral, bilateral, mask = edge_Mask)

            plt.figure(figsize = (15, 8))
            plt.subplot(1, 2, 1)
            plt.yticks([]), plt.xticks([])
            plt.title("Original Image")
            plt.imshow(img)
            plt.subplot(1, 2, 2)
            plt.yticks([]), plt.xticks([])
            plt.title("Cartoon Image")
            plt.imshow(cartoon)
            plt.show()

interactive(children=(Dropdown(description='style', index=1, options=('detail', 'pencil', 'cartoon'), value='p…